In [ ]:
!pip install openai
!pip install sentence-transformers
!pip install langchain pypdf langchain-openai #tiktoken chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.

In [ ]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

# RAG

In [ ]:
!pip install llama-index --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

In [ ]:
!wget https://www.goldmansachs.com/pdfs/insights/podcasts/episodes/ai-tom-acemoglu-covello/transcript.pdf

--2024-08-08 23:20:45--  https://www.goldmansachs.com/pdfs/insights/podcasts/episodes/ai-tom-acemoglu-covello/transcript.pdf
Resolving www.goldmansachs.com (www.goldmansachs.com)... 23.222.163.181
Connecting to www.goldmansachs.com (www.goldmansachs.com)|23.222.163.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘transcript.pdf’

transcript.pdf          [ <=>                ] 133.94K  --.-KB/s    in 0.08s   

2024-08-08 23:20:46 (1.65 MB/s) - ‘transcript.pdf’ saved [137153]



In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')

# Routing

# Routing

In [ ]:
# Import necessary classes from the llama_index package
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, SummaryIndex
from llama_index.core import Settings

# Read documents from the specified directory and load a specific document, "report.pdf".
documents = SimpleDirectoryReader("./").load_data("transcript.pdf")

# initialize settings (set chunk size)
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Create a VectorStoreIndex object from the documents. This will involve processing the documents
# and creating a vector representation for each of them, suitable for semantic searching.
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()


Loading files: 100%|██████████| 1/1 [00:01<00:00,  1.04s/file]


In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to the data source",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to the data source",
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)


In [ ]:
response = query_engine.query("Is AI overhyped?")
print(response)

AI may be considered overhyped due to concerns about the technology's ability to deliver on the significant investments being made in its development.


In [ ]:
response = query_engine.query("Summarize the document in 4 bullet points")
print(response)

- The document discusses the skepticism surrounding the cost-effectiveness and transformative potential of generative AI technology.
- Experts like Daron Acemoglu and Jim Covello question the current applications and cost efficiency of AI, highlighting uncertainties and limitations.
- There is a debate on whether the massive investment in AI by big tech companies will lead to significant returns or if it will result in wasted spending.
- The document emphasizes the importance of monitoring the development of AI applications, corporate profits, and the emergence of killer applications to assess the true impact and potential of AI investments.


# Sub Question Query Engine

In [ ]:
# Import necessary classes and modules from llama_index.core and llama_index.core.tools
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import Settings
from IPython.display import display, Markdown

# Load the data from a PDF file located in the current directory using SimpleDirectoryReader
# The load_data method reads the content of the file "transcript.pdf"
documents = SimpleDirectoryReader("./").load_data("transcript.pdf")

# Build a VectorStoreIndex from the loaded documents
# This index will allow efficient querying of the document content
vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True,  # Enable asynchronous processing for faster performance
).as_query_engine()  # Convert the index to a query engine

# Define a list of query engine tools, each with its own metadata
# This setup is necessary for the SubQuestionQueryEngine
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,  # Use the vector_query_engine built above
        metadata=ToolMetadata(
            name="documents",  # Name of the tool
            description="Report",  # Description of the tool
        ),
    ),
]

# Create an instance of SubQuestionQueryEngine using the default settings
# This query engine can handle sub-questions and use the provided tools for querying
query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,  # Provide the list of query engine tools
    use_async=True,  # Enable asynchronous processing
)

# Query the SubQuestionQueryEngine with the question "Is AI overhyped?"
response = query_engine.query(
    "Is AI overhyped?"
)

# Print the response from the query
print(response.response)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  2.13file/s]


**********
Trace: index_construction
    |_embedding -> 0.406346 seconds
**********
Generated 2 sub questions.
[documents] Q: What are the current trends in AI adoption?
[documents] Q: What are the key challenges in AI implementation?
[documents] A: The current trends in AI adoption indicate a growing interest and investment in the potential of generative AI to automate work tasks, increase productivity, and boost GDP growth. However, there are also concerns about the high costs associated with developing and implementing AI technology, as well as uncertainties regarding the actual impact and effectiveness of AI in the near future.
[documents] A: The key challenges in AI implementation include the significant amount of money being invested in developing AI infrastructure and applications without a clear understanding of what trillion-dollar problem AI is meant to solve. Additionally, the concern arises from the fact that AI is an expensive solution intended to replace low-cost labor, w

In [ ]:
display(Markdown(response.response))

The information provided suggests that there is a significant level of interest and investment in AI, with growing expectations about its potential benefits. However, there are also concerns highlighted regarding the high costs associated with AI development, uncertainties about its actual impact, and challenges related to understanding the specific problems AI is meant to solve.

# Calling OpenAI AssistantAPI (Code interpreter)

In [ ]:
from llama_index.agent.openai import OpenAIAssistantAgent

agent = OpenAIAssistantAgent.from_new(
    name="Python agent",
    openai_tools=[{"type": "code_interpreter"}],
    instructions="You are an expert at writing python code to solve problems.",
    verbose=True
)

response = agent.chat(
    """Generate code to answer the following question:
    How much is the us population likely to grow to by 2030?
    Return and answer and the code used."""
)

**********
Trace: chat
    |_agent_step -> 18.860508 seconds
**********


In [ ]:
print(str(response))

Based on the assumption of a 0.5% average annual growth rate and a starting population of 331 million in 2021, the projected U.S. population by 2030 is approximately 346.2 million people.

Please note that this is a simplistic model and real demographic projections would take into account other factors such as immigration, birth rates, death rates, and policy changes which can significantly affect population growth. For a more accurate projection, one would refer to the latest projections from sources like the U.S. Census Bureau or the United Nations.

Here is the code that was used to estimate the population:

```python
current_population = 331_000_000  # U.S. population in 2021
annual_growth_rate = 0.005  # Average annual growth rate
years_to_project = 2030 - 2021  # Number of years to project

# Calculate projected population by 2030
projected_population = current_population * (1 + annual_growth_rate) ** years_to_project
projected_population
```


In [ ]:
display(Markdown(response.response))

Based on the assumption of a 0.5% average annual growth rate and a starting population of 331 million in 2021, the projected U.S. population by 2030 is approximately 346.2 million people.

Please note that this is a simplistic model and real demographic projections would take into account other factors such as immigration, birth rates, death rates, and policy changes which can significantly affect population growth. For a more accurate projection, one would refer to the latest projections from sources like the U.S. Census Bureau or the United Nations.

Here is the code that was used to estimate the population:

```python
current_population = 331_000_000  # U.S. population in 2021
annual_growth_rate = 0.005  # Average annual growth rate
years_to_project = 2030 - 2021  # Number of years to project

# Calculate projected population by 2030
projected_population = current_population * (1 + annual_growth_rate) ** years_to_project
projected_population
```

In [ ]:
population_2023 = 332_000_000  # 332 million

# Assumed average annual growth rate
annual_growth_rate = 0.7 / 100  # 0.7% growth rate

# Number of years from 2023 to 2030
years = 2030 - 2023

# Calculate the projected population for 2030
population_2030 = population_2023 * ((1 + annual_growth_rate) ** years)
population_2030

348613641.67707187

# Agents w/CrewAI


In [ ]:
# Notebook here: https://colab.research.google.com/drive/1IiFpMCwQtIIwdbF03tGUIgFeoPcezrsq#scrollTo=Q7VI8mLQRhfX